In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdmetrics.reports.single_table import DiagnosticReport
from sdv.evaluation.single_table import evaluate_quality
from sklearn.impute import SimpleImputer

In [2]:
# Load the Breast Cancer dataset from scikit-learn

real_data =  new_df = pd.read_csv('Data/patient_features_early_stage.csv')
# real_data = pd.DataFrame(data.data, columns=data.feature_names)

In [3]:
print("Real Data Sample:")
real_data.head()

Real Data Sample:


,diagnosis_stage_t,diagnosis_stage_n,diagnosis_stage_m,chemotherapy@t1_type,surgery@t1_procedure1,surgery@t1_perioperative_death,age,race,tumor_stage,histology_grade,...,surgery@t1_resection_grade,surgery@t1_t_stage,surgery@t1_n_stage,surgery@t1_pathological_response,adeno_histology,mesothelioma,carcinom,small_cell_carcinom,biomarker@diagnosis_type,biomarker@diagnosis_EGFR_negative
0,T2a,N1,M0,Adjuvant chemotherapy,Left inferior lobectomy,False,68.0,Caucasian,IIA,NaN,...,R0,T2a,N1,NaN,NaN,NaN,NaN,NaN,NaN,Yes
1,T1b,N0,M0,QT intravenous,Right inferior lobectomy,False,63.0,Caucasian,IA,Poor diferenciated,...,R0,T1b,N0,NaN,Unknown,NaN,NaN,NaN,NaN,Yes
2,T1b,N0,M0,QT intravenous,Right superior lobectomy,False,66.0,Caucasian,IA,NaN,...,R0,T1b,N0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T3,N0,M0,NaN,NaN,NaN,81.0,Caucasian,IIB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T1a,N0,M0,QT intravenous,Right superior lobectomy,False,63.0,Caucasian,IA,Non specified,...,R0,T1a,N2,NaN,Acinar,NaN,NaN,NaN,NaN,Yes


In [4]:
# Define the metadata for the dataset
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=real_data)

# Initialize the GaussianCopulaSynthesizer with metadata
synthesizer = GaussianCopulaSynthesizer(metadata)

# Fit the synthesizer to the real data
synthesizer.fit(real_data)
# Generate synthetic data based on the real data
synthetic_data = synthesizer.sample(len(real_data))

/Users/mohtim/opt/anaconda3/envs/py37/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)


In [5]:
# Display the first few rows of the synthetic data
print("\nSynthetic Data Sample:")
synthetic_data.head()


Synthetic Data Sample:


,diagnosis_stage_t,diagnosis_stage_n,diagnosis_stage_m,chemotherapy@t1_type,surgery@t1_procedure1,surgery@t1_perioperative_death,age,race,tumor_stage,histology_grade,...,surgery@t1_resection_grade,surgery@t1_t_stage,surgery@t1_n_stage,surgery@t1_pathological_response,adeno_histology,mesothelioma,carcinom,small_cell_carcinom,biomarker@diagnosis_type,biomarker@diagnosis_EGFR_negative
0,T1a,N1,M0,QT intravenous,Left penumonectomy,False,56.0,Caucasian,IIA,Moderately differentiated,...,R0,T3,NaN,NaN,Lepidic,NaN,NaN,NaN,EGFR,Yes
1,T2b,N0,M0,Adjuvant chemotherapy,NaN,NaN,78.0,Caucasian,IB,Undifferentiated,...,R0,Tx,N0,NaN,Others,NaN,NaN,NaN,EGFR,NaN
2,T2a,N1,M0,Adjuvant chemotherapy,Left superior lobectomy,False,51.0,Caucasian,IA,Non specified,...,R0,T2a,N1,NaN,Papillary,NaN,NaN,NaN,NaN,NaN
3,T3,N1,M0,NaN,Left superior lobectomy,False,81.0,Caucasian,IIB,NaN,...,R0,T1b,N1,NaN,Unknown,NaN,NaN,NaN,NaN,Yes
4,T1a,N1,M0,Adjuvant chemotherapy,NaN,False,52.0,Latin,IIB,Poor diferenciated,...,R0,NaN,N0,NaN,NaN,NaN,NaN,NaN,NaN,Yes


In [6]:
# Compare summary statistics between real and synthetic data
real_stats = real_data.describe()
synthetic_stats = synthetic_data.describe()

# Print the summary statistics side by side
print("\nReal Data Statistics:")
real_stats




Real Data Statistics:


,age,chemotherapy@t1_start_time,radiotherapy@t1_duration_days,surgery@t1_time,relapse?,id,nb_cig_packs_year,nb_cigs_day,family_lung_cancer,family_other_cancer,diagnosis_ecog,radiotherapy@t1_dose,radiotherapy@t1_fractioning,surgery@t1_pathological_response,mesothelioma,carcinom,small_cell_carcinom
count,1344.000000,625.000000,51.000000,1248.000000,1348.000000,1.348000e+03,962.000000,919.000000,1348.000000,1348.000000,1345.000000,49.000000,45.000000,0.0,0.0,0.0,0.0
mean,65.735119,10.276800,37.392157,3.019231,0.364243,3.190765e+06,53.655925,26.742111,0.139466,0.558605,0.426766,51.122449,427.706667,NaN,NaN,NaN,NaN
std,9.360557,17.825504,14.204335,5.987526,0.481396,3.748277e+06,30.442165,14.265605,0.455748,1.001991,0.590750,13.626923,917.838909,NaN,NaN,NaN,NaN
min,31.000000,0.000000,0.000000,0.000000,0.000000,1.006930e+05,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,1.800000,NaN,NaN,NaN,NaN
25%,60.000000,3.000000,33.000000,1.000000,0.000000,5.084408e+05,35.000000,20.000000,0.000000,0.000000,0.000000,50.000000,200.000000,NaN,NaN,NaN,NaN
50%,66.000000,4.000000,41.000000,2.000000,0.000000,1.315317e+06,50.000000,20.000000,0.000000,0.000000,0.000000,55.000000,200.000000,NaN,NaN,NaN,NaN
75%,73.000000,8.000000,45.500000,3.000000,1.000000,5.104005e+06,68.250000,40.000000,0.000000,1.000000,1.000000,60.000000,225.000000,NaN,NaN,NaN,NaN
max,118.000000,163.000000,69.000000,124.000000,1.000000,1.570639e+07,252.000000,120.000000,5.000000,7.000000,4.000000,70.000000,6000.000000,NaN,NaN,NaN,NaN


In [7]:
print("\nSynthetic Data Statistics:")
synthetic_stats


Synthetic Data Statistics:


,age,chemotherapy@t1_start_time,radiotherapy@t1_duration_days,surgery@t1_time,relapse?,id,nb_cig_packs_year,nb_cigs_day,family_lung_cancer,family_other_cancer,diagnosis_ecog,radiotherapy@t1_dose,radiotherapy@t1_fractioning,surgery@t1_pathological_response,mesothelioma,carcinom,small_cell_carcinom
count,1341.000000,616.00000,42.000000,1263.000000,1348.000000,1.348000e+03,980.000000,954.000000,1348.000000,1348.000000,1343.000000,54.000000,49.000000,0.0,0.0,0.0,0.0
mean,65.680089,10.76461,37.333333,2.783056,0.080119,3.661498e+06,53.802041,26.461216,0.718843,0.368694,0.016381,50.370370,412.220408,NaN,NaN,NaN,NaN
std,9.506626,8.31737,2.960444,3.513226,0.271578,3.774356e+06,25.325695,11.611635,1.337163,1.097936,0.132722,2.234348,63.641378,NaN,NaN,NaN,NaN
min,34.000000,0.00000,32.000000,0.000000,0.000000,1.006940e+05,4.000000,4.000000,0.000000,0.000000,0.000000,45.000000,279.100000,NaN,NaN,NaN,NaN
25%,59.000000,4.00000,35.000000,0.000000,0.000000,5.647405e+05,36.000000,18.000000,0.000000,0.000000,0.000000,49.000000,366.100000,NaN,NaN,NaN,NaN
50%,66.000000,9.00000,37.000000,1.000000,0.000000,2.239300e+06,50.500000,25.000000,0.000000,0.000000,0.000000,51.000000,410.100000,NaN,NaN,NaN,NaN
75%,72.000000,15.00000,39.000000,4.000000,0.000000,5.910732e+06,67.250000,33.000000,1.000000,0.000000,0.000000,52.000000,456.500000,NaN,NaN,NaN,NaN
max,99.000000,52.00000,44.000000,22.000000,1.000000,1.570639e+07,159.000000,78.000000,5.000000,7.000000,2.000000,54.000000,576.700000,NaN,NaN,NaN,NaN


In [8]:
common_columns = real_data.columns.intersection(synthetic_data.columns)
print ("columns of the real data",len(real_data.columns ))
print ("columns of the synthetic data",len(synthetic_data.columns))

columns of the real data 76
columns of the synthetic data 76


In [9]:
from sdmetrics.single_table import NewRowSynthesis

NewRowSynthesis.compute(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata
)

1.0

It looks like 100% of the rows in the synthetic data are new -- meaning that there are no exact copies of the real data!

In [10]:
synthetic_data.to_csv('synthetic_relapse_data_gaussian.csv', index=False)  # Set index=False to avoid writing row numbers